<a href="https://colab.research.google.com/github/minzzii-kim/machine-learing/blob/main/tensorflow_note_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# [이미지 분류기 만들기]
# Type A : IDG
# Type B : tfds

# [Type A] - IDG를 활용해 Image Augmentation
# fn {flow_from_directory} : 
# 로컬에 있는 이미지파일의 루트경로만 지정해주면 알아서
# 지정된 타겟사이즈만큼 이미지를 잘라서 batch 단위로 메모리에 올리고,
# subdirectory 단위로 이미지들을 라벨링한다.
# arg {class_mode} : 마지막 출력층 형태를 보고 결정
# Dense(1, activation='sigmoid') => class_mode='binary'
# Dense(2, activation='softmax') => class_mode='categorical'

In [ ]:
  # [특성추출]
  
  # CNN (Convolutional Neural Network, 합성곱 신경망)
  # 원본이미지(nxn) + CNN_필터_1(3x3) -> feature_map_1 ((n-2)x(n-2))
  # 원본이미지(nxn) + CNN_필터_2(3x3) -> feature_map_2 ((n-2)x(n-2))
  # 원본이미지(nxn) + CNN_필터_3(3x3) -> feature_map_3 ((n-2)x(n-2))
  # .....
  # => 필터 갯수만큼 다양한 feature_map 생성
  # Conv2D(n, (f,f), activation='relu') ; n : 필터갯수, f : 필터사이즈

  # [Pooling Layer] - 연산량 줄여줌
  # MaxPooling2D Layer
  # MaxPooling2D(n, n) -> 원본사이즈 (가로 * 1/n) * (세로 * 1/n) 만큼 크기가 줄어든다.

  # => 이미지 특성추출시 Conv2D + MaxPooling2D 조합으로 많이사용


In [8]:
from urllib.request import urlretrieve
from zipfile import ZipFile

import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urlretrieve(url, 'rps.zip')
# zip_ref = ZipFile('rps.zip', 'r')
# zip_ref.extractall('tmp/')
# zip_ref.close()

with ZipFile('rps.zip','r') as zf:
  zf.extractall('tmp/')


In [4]:
# ImageDataGenerator 를 활용해 이미지데이터 전처리

# IDG spec 정의
IDG = ImageDataGenerator(
    rescale=1./255., # 정규화
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

ROOT_DIR = 'tmp/rps/'
train_data = IDG.flow_from_directory(
    ROOT_DIR,
    batch_size=64, #128
    target_size=(150,150),
    class_mode='categorical',
    subset='training'
)
valid_data = IDG.flow_from_directory(
    ROOT_DIR,
    batch_size=64, #128
    target_size=(150,150),
    class_mode='categorical',
    subset='validation'
)



Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.


In [13]:
# 모델정의 및 생성

model = Sequential([
  Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
  MaxPooling2D(2,2),
  Conv2D(64, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Flatten(),
  Dropout(0.5),
  Dense(512, activation='relu'),
  # Dense(32, activation='relu'),
  # Dense(16, activation='relu'),
  Dense(3, activation='softmax'),
])

tmp_file = 'tmp_checkpoint.ckpt'
checkpoint_cb = ModelCheckpoint(tmp_file, 
                                monitor='val_loss',
                                verbose=1, 
                                save_best_only=True,
                                save_weights_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'] )


In [14]:
# 모델학습

model.fit(train_data, 
          validation_data = (valid_data), 
          epochs=25, 
          callbacks=[checkpoint_cb])

Epoch 1/20
32/32 [==============================] - ETA: 0s - loss: 1.1114 - acc: 0.3621
Epoch 00001: val_loss improved from inf to 1.08437, saving model to tmp_checkpoint.ckpt
32/32 [==============================] - 24s 731ms/step - loss: 1.1114 - acc: 0.3621 - val_loss: 1.0844 - val_acc: 0.3333
Epoch 2/20
32/32 [==============================] - ETA: 0s - loss: 0.9931 - acc: 0.4871
Epoch 00002: val_loss improved from 1.08437 to 0.89341, saving model to tmp_checkpoint.ckpt
32/32 [==============================] - 23s 709ms/step - loss: 0.9931 - acc: 0.4871 - val_loss: 0.8934 - val_acc: 0.5377
Epoch 3/20
32/32 [==============================] - ETA: 0s - loss: 0.7524 - acc: 0.6419
Epoch 00003: val_loss improved from 0.89341 to 0.85120, saving model to tmp_checkpoint.ckpt
32/32 [==============================] - 23s 705ms/step - loss: 0.7524 - acc: 0.6419 - val_loss: 0.8512 - val_acc: 0.6091
Epoch 4/20
32/32 [==============================] - ETA: 0s - loss: 0.5372 - acc: 0.7728
Epoch 

In [15]:
model.load_weights(tmp_file)

model.evaluate(valid_data)

8/8 [==============================] - 4s 522ms/step - loss: 0.5298 - acc: 0.8075


[0.5297673344612122, 0.807539701461792]